# micrograd exercies

## section 1: derivatives

In [1]:
#https://www.youtube.com/watch?v=VMj-3S1tku0&t=7096s
#https://colab.research.google.com/drive/1FPTx1RXtBfc4MaTkf7viZZD4U2F9gtKN?usp=sharing#scrollTo=qXaH59eL9zxf

from math import sin, cos

def f(a, b, c):
    return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [3]:
# write the function df that returns the analytical gradient of f

def gradf(a, b, c):
    #return [0, 0, 0] # todo, return [df/da, df/db, df/dc]
    return [-3.0*a**2 - 0.5*a**-0.5, 3*cos(3*b) + 2.5*b**1.5, c**-2]

# expected answeris the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
mine = gradf(2, 3, 4)
for dim in range(3):
    feedback = ':)' if abs(mine[dim] - ans[dim]) < 1e-5 else ':('
    print(f"{feedback} for dim {dim}: expected {ans[dim]}, mine returns {mine[dim]}") 

:) for dim 0: expected -12.353553390593273, mine returns -12.353553390593273
:) for dim 1: expected 10.25699027111255, mine returns 10.25699027111255
:) for dim 2: expected 0.0625, mine returns 0.0625


## section 2: support for softmax

In [1]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out
  
  def __radd__(self, other): #other + self
        return self + other
  
  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO
  # ------
  def exp(self):
    x = self.data
    out = Value(exp(x), (self,), 'exp')

    def _backward():
      self.grad += out.data * out.grad

    out._backward = _backward

    return out
  
  def log(self):
    x = self.data
    out = Value(log(x), (self,), 'log')

    def _backward():
      self.grad += x**-1 * out.grad

    out._backward = _backward

    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
        self.grad += other.data * out.grad #local derivative * gradient
        other.grad += self.data * out.grad #local derivative * gradient

    out._backward = _backward

    return out

  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * self.data ** (other - 1) * out.grad 
    out._backward = _backward

    return out

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self):
    return self * -1    


  def backward(self): # exactly as in video  
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [2]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  print(counts)
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


[Value(data=1.0), Value(data=20.085536923187668), Value(data=0.1353352832366127), Value(data=2.718281828459045)]
2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [46]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
m = torch.nn.Softmax(dim=0)
logits = torch.Tensor([0.0, 3.0, -2.0, 1.0]); logits.requires_grad = True
#logits = torch.Tensor([0.0]).double(); logits.requires_grad = True
output = m(logits)
output

tensor([0.0418, 0.8390, 0.0057, 0.1135], grad_fn=<SoftmaxBackward0>)

In [47]:
loss = -output[3].log()
loss.backward()
print(loss.data)

tensor(2.1755)


In [48]:
logits.grad

tensor([ 0.0418,  0.8390,  0.0057, -0.8865])

In [49]:
sum(output)

tensor(1.0000, grad_fn=<AddBackward0>)